In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "3"  # Set the GPUs to use
import torch
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Current cuda device: 0
Count of using GPUs: 1


In [2]:
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
)

import torch, copy, os
from module.score import evaluation_f1
from module.load_json import *
from module.maps import *
from module.args import print_torch_info
from module.inference import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print_torch_info()

torch.__version__: 1.7.1
torch.cuda.is_available(): True
NGPU: 1


# Paths and Modes

In [3]:
EVAL_MODE = False

RESULT_ID = 'uncleaned_v21_run_1_ensemble'

ACD_1_CHECKPOINT = 'training_results/uncleaned_v21_run_2/acd_klue_roberta_base_mlm_fine_tuned_uncleaned_v21_run_2/checkpoint-12500'
ACD_2_CHECKPOINT = 'training_results/uncleaned_v21_run_1/acd_monologg_koelectra_base_v3_discriminator_uncleaned_v21_run_1/checkpoint-12500'
ACD_3_CHECKPOINT = 'training_results/uncleaned_v21_run_4/acd_snunlp_kr_electra_discriminator_uncleaned_v21_run_4/checkpoint-12500'

ASC_1_CHECKPOINT = 'training_results/uncleaned_v22_run_1/asc_klue_roberta_base_mlm_fine_tuned_uncleaned_v22_run_1/checkpoint-1608'
ASC_2_CHECKPOINT = 'training_results/uncleaned_v22_run_1/asc_monologg_koelectra_base_v3_discriminator_uncleaned_v22_run_1/checkpoint-1608'
ASC_3_CHECKPOINT = 'training_results/uncleaned_v22_run_1/asc_snunlp_kr_electra_discriminator_uncleaned_v22_run_1/checkpoint-1608'

TEST_DATA_PATH = 'dataset/nikluge-sa-2022-test.jsonl'
EVAL_DATA_PATH = 'dataset/nikluge-sa-2022-dev.jsonl'

In [4]:
if EVAL_MODE == True:
    TEST_DATA_PATH = EVAL_DATA_PATH
test_data = jsonlload(TEST_DATA_PATH)
print('>>>>> >>>>> >>>>> ', TEST_DATA_PATH, ' <<<<< <<<<< <<<<<', '\n', sep='')

>>>>> >>>>> >>>>> dataset/nikluge-sa-2022-test.jsonl <<<<< <<<<< <<<<<



# Load Models and Tokenizers

In [5]:
acd_model_1 = AutoModelForSequenceClassification.from_pretrained(ACD_1_CHECKPOINT)
acd_model_2 = AutoModelForSequenceClassification.from_pretrained(ACD_2_CHECKPOINT)
acd_model_3 = AutoModelForSequenceClassification.from_pretrained(ACD_3_CHECKPOINT)
acd_tokenizer_1 = AutoTokenizer.from_pretrained(ACD_1_CHECKPOINT)
acd_tokenizer_2 = AutoTokenizer.from_pretrained(ACD_2_CHECKPOINT)
acd_tokenizer_3 = AutoTokenizer.from_pretrained(ACD_3_CHECKPOINT)

asc_model_1 = AutoModelForSequenceClassification.from_pretrained(ASC_1_CHECKPOINT)
asc_model_2 = AutoModelForSequenceClassification.from_pretrained(ASC_2_CHECKPOINT)
asc_model_3 = AutoModelForSequenceClassification.from_pretrained(ASC_3_CHECKPOINT)
asc_tokenizer_1 = AutoTokenizer.from_pretrained(ASC_1_CHECKPOINT)
asc_tokenizer_2 = AutoTokenizer.from_pretrained(ASC_2_CHECKPOINT)
asc_tokenizer_3 = AutoTokenizer.from_pretrained(ASC_3_CHECKPOINT)

In [6]:
def acd_ensemble(form, acd_pair):
    acd_model_1.to(device)
    acd_model_2.to(device)
    acd_model_3.to(device)
    acd_model_1.eval()
    acd_model_2.eval()
    acd_model_3.eval()

    acd_encoded_1 = acd_tokenizer_1(form, acd_pair, truncation=True, return_tensors="pt")
    acd_encoded_1 = {k:v.to(device) for k,v in acd_encoded_1.items()}

    acd_encoded_2 = acd_tokenizer_2(form, acd_pair, truncation=True, return_tensors="pt")
    acd_encoded_2 = {k:v.to(device) for k,v in acd_encoded_2.items()}

    acd_encoded_3 = acd_tokenizer_3(form, acd_pair, truncation=True, return_tensors="pt")
    acd_encoded_3 = {k:v.to(device) for k,v in acd_encoded_3.items()}

    with torch.no_grad():
        acd_outputs_1 = acd_model_1(**acd_encoded_1)['logits']
        acd_outputs_2 = acd_model_2(**acd_encoded_2)['logits']
        acd_outputs_3 = acd_model_3(**acd_encoded_3)['logits']

    acd_outputs = acd_outputs_1 + acd_outputs_2 + acd_outputs_3
    
    return acd_outputs

In [7]:
def asc_ensemble(form, asc_pair):
    asc_model_1.to(device)
    asc_model_2.to(device)
    asc_model_3.to(device)
    asc_model_1.eval()
    asc_model_2.eval()
    asc_model_3.eval()

    asc_encoded_1 = asc_tokenizer_1(form, asc_pair, truncation=True, return_tensors="pt")
    asc_encoded_1 = {k:v.to(device) for k,v in asc_encoded_1.items()}

    asc_encoded_2 = asc_tokenizer_2(form, asc_pair, truncation=True, return_tensors="pt")
    asc_encoded_2 = {k:v.to(device) for k,v in asc_encoded_2.items()}

    asc_encoded_3 = asc_tokenizer_3(form, asc_pair, truncation=True, return_tensors="pt")
    asc_encoded_3 = {k:v.to(device) for k,v in asc_encoded_3.items()}

    with torch.no_grad():
        asc_outputs_1 = asc_model_1(**asc_encoded_1)['logits']
        asc_outputs_2 = asc_model_2(**asc_encoded_2)['logits']
        asc_outputs_3 = asc_model_3(**asc_encoded_3)['logits']

    asc_outputs = asc_outputs_1 + asc_outputs_2 + asc_outputs_3
    
    return asc_outputs

# Inference

In [8]:
for entity_property_pair in entity_property_pairs:
    num_of_ep_pairs = len(entity_property_pair)
    RESULT_SAVE_NAME = f'{RESULT_ID}_num_of_ep_pairs_{num_of_ep_pairs}.json'
    
    pred_data = inference_m_ensemble(acd_ensemble, asc_ensemble, copy.deepcopy(test_data), entity_property_pair)
    
    if EVAL_MODE == False:
        save_path = './'

        jsondump(pred_data, os.path.join(save_path, RESULT_SAVE_NAME))
        pred_data = jsonload(os.path.join(save_path, RESULT_SAVE_NAME))
    
    print(RESULT_SAVE_NAME)    
    print(len(test_data), len(pred_data))
    
    if EVAL_MODE == True:
        print('INFERENCE DATA: ', TEST_DATA_PATH)

        print('EVAL_MODE :', EVAL_MODE)

        result = evaluation_f1(test_data, pred_data)
        print(list(result.items())[0])
        print(list(result.items())[1])
    break

['본품#가격', '본품#다양성', '본품#디자인', '본품#인지도', '본품#일반', '본품#편의성', '본품#품질', '브랜드#가격', '브랜드#디자인', '브랜드#인지도', '브랜드#일반', '브랜드#품질', '제품 전체#가격', '제품 전체#다양성', '제품 전체#디자인', '제품 전체#인지도', '제품 전체#일반', '제품 전체#편의성', '제품 전체#품질', '패키지/구성품#가격', '패키지/구성품#다양성', '패키지/구성품#디자인', '패키지/구성품#일반', '패키지/구성품#편의성', '패키지/구성품#품질']


  0%|          | 2/2127 [00:15<3:51:05,  6.52s/it]

동양인 피부톤과 잘 어울리고 우아한 분위기를 풍긴다네?


  0%|          | 6/2127 [00:21<1:18:50,  2.23s/it]

생각보다 드라큘라 백작같지 않던데?


...

아이들과 어른들 얼굴 사이즈가 틀리기 때문에 같이 사용이 가능할까? 싶었는데 사진처럼 폴딩이 들어간 디자인이라 가운데 부분이 부드럽게 잘 접혀요.


100%|█████████▉| 2126/2127 [56:19<00:01,  1.49s/it]

본체부터 케이블, 설명서까지 깔끔하게 정리되어 보관이 가능하니 더더 맘에 쏙 들어요.


100%|██████████| 2127/2127 [56:20<00:00,  1.59s/it]

사이즈가 컴펙트하고 무게감도 부담없어 가방에 쏙 넣고 다니기도 좋아요.
uncleaned_v21_run_1_ensemble_num_of_ep_pairs_25.json
2127 2127
